![1](1.png)

![2](2.png)

![3](3.png)

![4](4.png)

In [1]:
from pyomo.environ import *


# Create a new instance of a ConcreteModel
model = ConcreteModel()

days_revenue = {'Thursday': 1, 'Friday': 1.5, 'Saturday': 2, 'Sunday': 0.5}
days = ['Thursday','Friday','Saturday','Sunday']
games = {('BOS', 'LAL'): 2, ('ATL', 'LAC'): 0.75,
                  ('POR', 'MIL'): 1.25, ('DEN', 'WAS'): 0.6,
                  ('UTA', 'BKN'): 1.5, ('IND', 'DAL'): 1.1,
                  ('CHA', 'NOP'): 1.2, ('NYK', 'HOU'): 0.6,
                  ('TOR', 'PHX'): 1, ('CHI', 'CLE'): 0.45,
                  ('MIA', 'ORL'): 1.1, ('DET', 'SAS'): 0.5,
                  ('SAC', 'GSW'): 1.25, ('MEM', 'OKC'): 0.75,
                  ('PHI', 'MIN'): 1.3, ('TOR', 'LAL'): 1.9,
                  ('PHX', 'GSW'): 1.4, ('LAC', 'SAC'): 0.9,
                  ('UTA', 'WAS'): 1.1, ('NYK', 'SAS'): 0.8}
teams = ['BOS', 'ATL', 'POR', 'DEN', 'UTA', 'IND', 'CHA', 'NYK', 'TOR', 'CHI', 'MIA', 'DET', 'SAC', 'MEM', 'PHI', 'PHX', 'LAC','LAL', 'WAS', 'SAS', 'GSW', 'NOP', 'HOU', 'OKC', 'MIN']

model.x = Var(days,games.keys(),domain = Binary)

#Objective : maximize revenue 
def revenue_rule(model):
    return sum(model.x[day, game] * days_revenue[day] * games[game] for day in days for game in games)
model.revenue = Objective(rule=revenue_rule, sense=maximize)


In [2]:
# Add the constraint for each team everyday
def team_play_once_rule(model, team, day):
        return sum(model.x[day, game] for game in games if team in game) <= 1
model.team_play_once = Constraint(teams, days, rule=team_play_once_rule)

# Add the constraint for each day
def minimum_rating_rule(model, day):
    return sum(model.x[day, game] for game in games if games[game] >= 1.25) >= 1
model.minimum_rating = Constraint(days, rule=minimum_rating_rule)

# Add LA constraint 
def LA_rule(model, day):
        return sum(model.x[day,game] for game in games if game[1] == 'LAL' or game[1] == 'LAC') <= 1
model.LA = Constraint(days, rule=LA_rule)

# Add super bowl sunday constraint:
def superbowl_rule(model):
    return sum(model.x[day, game] for day in days if day == 'Sunday' for game in games if games[game] > 1.4) <= 0
model.superbowl = Constraint(rule=superbowl_rule)
    
# Add NY UTA constraint:
def NY_UTA_rule(model):
    return sum(model.x[day,game] for game in games for team in game if team == "NY" or team == "UTA" for day in days if day != "Thursday" and day != "Friday") <= 0
model.NY_UTA = Constraint(rule=NY_UTA_rule)  
    
# Add games slot constraints:
game_slots = {'Thursday': 5, 'Friday': 6, 'Saturday': 6, 'Sunday': 3}

def daily_game_limit_rule(model, day):
        return sum(model.x[day, game] for game in games) == game_slots[day]
model.daily_game_limit = Constraint(days, rule=daily_game_limit_rule)

# Add each game only one contraints:
def games_rule(model, team1, team2):
    return sum(model.x[ day,(team1, team2)] for day in days) <= 1
model.games = Constraint(games.keys(), rule=games_rule)

In [3]:
SolverFactory('gurobi').solve(model).write()
print('\nMax Revenue = ', model.revenue())

for day in days:
    daily_revenue = 0
    print(f"Day: {day}")
    for game in games.keys():
        if model.x[day, game].value == 1:
            game_revenue = days_revenue[day] * games[game]
            print(f"Game: {game[0]} vs {game[1]}, Rating {games[game]}")
            daily_revenue += game_revenue
    print(f"Total Revenue for {day}: {daily_revenue}\n")

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1
  Lower bound: 31.950000000000006
  Upper bound: 31.950000000000006
  Number of objectives: 1
  Number of constraints: 134
  Number of variables: 80
  Number of binary variables: 80
  Number of integer variables: 80
  Number of continuous variables: 0
  Number of nonzeros: 347
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was

In [5]:
# Add PHX LAL back to back constraint:
def PHX_LAL_back_to_back_constraint_1(model, team):
    if team in ['PHX', 'LAL']:
        return (sum(model.x[day, game] for day in days if day =='Thursday' for game in games if team in game) +
                sum(model.x[day , game] for day in days if day == 'Friday' for game in games if team in game)) <= 1
    else:
        return Constraint.Skip
model.PHX_LAL_back_to_back_1 = Constraint(teams, rule=PHX_LAL_back_to_back_constraint_1)


def PHX_LAL_back_to_back_constraint_2(model, team):
    if team in ['PHX', 'LAL']:
        return (sum(model.x[day, game] for day in days if day =='Friday' for game in games if team in game) +
                sum(model.x[day , game] for day in days if day == 'Saturday' for game in games if team in game)) <= 1
    else:
        return Constraint.Skip
model.PHX_LAL_back_to_back_2 = Constraint(teams, rule=PHX_LAL_back_to_back_constraint_2)


def PHX_LAL_back_to_back_constraint_3(model, team):
    if team in ['PHX', 'LAL']:
        return (sum(model.x[day, game] for day in days if day =='Saturday' for game in games if team in game) +
                sum(model.x[day , game] for day in days if day == 'Sunday' for game in games if team in game)) <= 1
    else:
        return Constraint.Skip
model.PHX_LAL_back_to_back_3 = Constraint(teams, rule=PHX_LAL_back_to_back_constraint_3)

In [6]:
SolverFactory('gurobi').solve(model).write()
print('\nMax Revenue = ', model.revenue())

for day in days:
    daily_revenue = 0
    print(f"Day: {day}")
    for game in games.keys():
        if model.x[day, game].value == 1:
            game_revenue = days_revenue[day] * games[game]
            print(f"Game: {game[0]} vs {game[1]}, Rating {games[game]}")
            daily_revenue += game_revenue
    print(f"Total Revenue for {day}: {daily_revenue}\n")

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1
  Lower bound: 31.275000000000006
  Upper bound: 31.275000000000006
  Number of objectives: 1
  Number of constraints: 140
  Number of variables: 80
  Number of binary variables: 80
  Number of integer variables: 80
  Number of continuous variables: 0
  Number of nonzeros: 371
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was